<a href="https://www.kaggle.com/code/albertobircoci/bird-vs-drone-prediction-cnn-lstm?scriptVersionId=231022234" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

def get_image_label_lists(images_dir, labels_dir):
    image_paths = []
    class_labels = []
    
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(images_dir, filename)
            label_filename = filename.replace('.jpg', '.txt')
            label_path = os.path.join(labels_dir, label_filename)
            
            class_id = 0
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 0:
                        tokens = lines[0].split()
                        if len(tokens) > 0:
                            try:
                                class_id = int(tokens[0])
                            except ValueError:
                                pass
            
            image_paths.append(img_path)
            class_labels.append(class_id)
    
    return image_paths, class_labels

train_images_dir = "/kaggle/input/bird-vs-drone/Dataset/train/images"
train_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/train/labels"

valid_images_dir = "/kaggle/input/bird-vs-drone/Dataset/valid/images"
valid_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/valid/labels"

test_images_dir = "/kaggle/input/bird-vs-drone/Dataset/test/images"
test_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/test/labels"

train_image_paths, train_class_labels = get_image_label_lists(train_images_dir, train_labels_dir)
valid_image_paths, valid_class_labels = get_image_label_lists(valid_images_dir, valid_labels_dir)
test_image_paths, test_class_labels   = get_image_label_lists(test_images_dir, test_labels_dir)

print(f"Number of training images: {len(train_image_paths)}")
print(f"Number of validation images: {len(valid_image_paths)}")
print(f"Number of test images: {len(test_image_paths)}")
train_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_class_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_image_paths, valid_class_labels))
test_ds  = tf.data.Dataset.from_tensor_slices((test_image_paths, test_class_labels))

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    return image, label

train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds  = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
batch_size = 32
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

Number of training images: 18323
Number of validation images: 1740
Number of test images: 889


In [2]:
model = models.Sequential([
    layers.InputLayer(input_shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=4
)

test_loss, test_acc = model.evaluate(test_ds)
print(f"[CNN Model] Test Accuracy: {test_acc * 100:.8f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       5,537,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,631,169 (21.48 MB)

 Trainable params: 5,631,169 (21.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 69s 105ms/step - accuracy: 0.9951 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 1.0000 - loss: 2.2651e-19 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 1.0000 - loss: 2.2764e-19 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - accuracy: 1.0000 - loss: 2.1451e-19 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.9012 - loss: 43.1684
[CNN Model] Test Accuracy: 89.98875022%


In [4]:
input_layer = tf.keras.Input(shape=(224, 224, 3))

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
shape = tf.keras.backend.int_shape(x)  # e.g. (None, H, W, Channels)
x = tf.keras.layers.Reshape((shape[1], shape[2] * shape[3]))(x)
x = tf.keras.layers.LSTM(128, return_sequences=False)(x)

x = tf.keras.layers.Dense(64, activation='relu')(x)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_lstm = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.summary()

history_lstm = model_lstm.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=4
)
test_loss, test_accuracy = model_lstm.evaluate(test_ds)
print(f"[CNN+LSTM Model] Test Loss: {test_loss:.4f}")
print(f"[CNN+LSTM Model] Test Accuracy: {test_accuracy * 100:.8f}%")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 56, 3584)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │       1,901,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,928,769 (7.36 MB)

 Trainable params: 1,928,769 (7.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 30s 45ms/step - accuracy: 0.9879 - loss: 0.0139 - val_accuracy: 1.0000 - val_loss: 3.4459e-06
Epoch 2/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 1.0000 - loss: 2.0362e-06 - val_accuracy: 1.0000 - val_loss: 7.2134e-07
Epoch 3/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - accuracy: 1.0000 - loss: 4.8000e-07 - val_accuracy: 1.0000 - val_loss: 2.7713e-07
Epoch 4/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 1.0000 - loss: 1.9618e-07 - val_accuracy: 1.0000 - val_loss: 1.3810e-07
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9012 - loss: 1.5936
[CNN+LSTM Model] Test Loss: 1.6149
[CNN+LSTM Model] Test Accuracy: 89.98875022%


In [5]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False
# Build the transfer learning model
model_transfer = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_transfer.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_transfer.summary()
history_transfer = model_transfer.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=4
)

test_loss, test_accuracy = model_transfer.evaluate(test_ds)
print(f"[Transfer Learning Model] Test Loss: {test_loss:.4f}")
print(f"[Transfer Learning Model] Test Accuracy: {test_accuracy * 100:.2f}%")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │          81,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,340,033 (8.93 MB)

 Trainable params: 82,049 (320.50 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 1.0000 - val_loss: 8.9872e-07
Epoch 2/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - accuracy: 1.0000 - loss: 6.0465e-09 - val_accuracy: 1.0000 - val_loss: 8.7401e-07
Epoch 3/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - accuracy: 1.0000 - loss: 5.7515e-09 - val_accuracy: 1.0000 - val_loss: 8.3690e-07
Epoch 4/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 1.0000 - loss: 5.6027e-09 - val_accuracy: 1.0000 - val_loss: 7.8924e-07
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - accuracy: 0.9012 - loss: 1.6439
[Transfer Learning Model] Test Loss: 1.6515
[Transfer Learning Model] Test Accuracy: 89.99%


In [3]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# === Step 1. Load image paths and labels ===
def get_image_label_lists(images_dir, labels_dir):
    image_paths = []
    class_labels = []
    
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(images_dir, filename)
            label_filename = filename.replace('.jpg', '.txt')
            label_path = os.path.join(labels_dir, label_filename)
            
            class_id = 0
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 0:
                        tokens = lines[0].split()
                        if len(tokens) > 0:
                            try:
                                class_id = int(tokens[0])
                            except ValueError:
                                pass
            
            image_paths.append(img_path)
            class_labels.append(class_id)
    
    return image_paths, class_labels

# Define directories for training, validation, and test sets.
train_images_dir = "/kaggle/input/bird-vs-drone/Dataset/train/images"
train_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/train/labels"

valid_images_dir = "/kaggle/input/bird-vs-drone/Dataset/valid/images"
valid_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/valid/labels"

test_images_dir = "/kaggle/input/bird-vs-drone/Dataset/test/images"
test_labels_dir = "/kaggle/input/bird-vs-drone/Dataset/test/labels"

# Get image paths and labels.
train_image_paths, train_class_labels = get_image_label_lists(train_images_dir, train_labels_dir)
valid_image_paths, valid_class_labels = get_image_label_lists(valid_images_dir, valid_labels_dir)
test_image_paths, test_class_labels   = get_image_label_lists(test_images_dir, test_labels_dir)

print(f"Number of training images: {len(train_image_paths)}")
print(f"Number of validation images: {len(valid_image_paths)}")
print(f"Number of test images: {len(test_image_paths)}")

# Create TensorFlow datasets from the paths and labels.
train_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_class_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_image_paths, valid_class_labels))
test_ds  = tf.data.Dataset.from_tensor_slices((test_image_paths, test_class_labels))

# === Step 2. Load and preprocess images ===
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds  = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batch the datasets (for validation and test, batching is fine as is).
batch_size = 32
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# === Step 3. Apply Data Augmentation on the Training Set ===
# Define the augmentation pipeline.
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

# Define an augmentation function for each image.
def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label

# Since train_ds is batched, unbatch it first so that each element is a single image.
train_ds = train_ds.unbatch()
train_ds = train_ds.shuffle(1000)  # Shuffle individual images again.
train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# === Step 4. Build, Train, and Evaluate the CNN Model ===
# Build a simple CNN model.
model = models.Sequential([
    layers.InputLayer(input_shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification.
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Train the model.
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=4
)

# Evaluate the model on the test dataset.
test_loss, test_acc = model.evaluate(test_ds)
print(f"[CNN Model with Augmentation] Test Accuracy: {test_acc * 100:.2f}%")


Number of training images: 18323
Number of validation images: 1740
Number of test images: 889


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       5,537,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,631,169 (21.48 MB)

 Trainable params: 5,631,169 (21.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
    573/Unknown 96s 158ms/step - accuracy: 0.9906 - loss: 0.0085

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


573/573 ━━━━━━━━━━━━━━━━━━━━ 99s 162ms/step - accuracy: 0.9906 - loss: 0.0085 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 94s 159ms/step - accuracy: 1.0000 - loss: 2.4779e-20 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 94s 160ms/step - accuracy: 1.0000 - loss: 2.1374e-20 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/4
573/573 ━━━━━━━━━━━━━━━━━━━━ 94s 160ms/step - accuracy: 1.0000 - loss: 5.2538e-20 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9012 - loss: 39.8795
[CNN Model with Augmentation] Test Accuracy: 89.99%
